# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770630644916                   -0.53    9.0         
  2   -2.772139553860       -2.82       -1.29    1.0    106ms
  3   -2.772169965191       -4.52       -2.71    1.0    132ms
  4   -2.772170721170       -6.12       -3.92    2.0    126ms
  5   -2.772170722954       -8.75       -4.57    1.0    111ms
  6   -2.772170723009      -10.26       -5.29    1.0    131ms
  7   -2.772170723015      -11.24       -5.70    2.0    129ms
  8   -2.772170723015      -12.36       -7.10    1.0    124ms
  9   -2.772170723015   +  -14.57       -7.31    2.0    142ms
 10   -2.772170723015      -13.88       -7.46    1.0    135ms
 11   -2.772170723015      -14.75       -8.65    1.0    119ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770859817446                   -0.52    9.0         
  2   -2.7

1.7735581146963935

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770706878369                   -0.53    8.0         
  2   -2.772045924487       -2.87       -1.30    1.0    151ms
  3   -2.772081486643       -4.45       -2.65    1.0    138ms
  4   -2.772083412748       -5.72       -3.83    2.0    154ms
  5   -2.772083417471       -8.33       -4.28    2.0    126ms
  6   -2.772083417800       -9.48       -5.35    1.0    133ms
  7   -2.772083417810      -10.97       -6.07    2.0    129ms
  8   -2.772083417811      -12.84       -6.88    2.0    147ms
  9   -2.772083417811   +  -14.40       -7.61    1.0    128ms
 10   -2.772083417811   +    -Inf       -8.44    2.0    150ms

Polarizability via ForwardDiff:       1.7725349619169557
Polarizability via finite difference: 1.7735581146963935
